# Implementing a CNN using Keras for given task

##### The CNN below involves a Sequential model with convolutional layers and fully connected layers with no skip connections

In [1]:
# Importing the required modules and their sub-modules

import os
import numpy as np
import pandas as pd
import pickle
from keras.layers import *
from keras.layers.core import *
from keras.models import Sequential

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

Using TensorFlow backend.


### I have implemented a sequential model which takes (32,32,1) images as input and predicts a label. representing any of the four classes as given. I observed some of the images and found them to be blurred, but didn't apply any deblurring technique due to my ignorance about it. 

#### 1) I have used conv layers with ReLU activation as I found it to give a decent performance over other activations.
#### 2) After every activation, the result is stabilized with Batch Normalization on the channel axis.
#### 3) There are two max-pooling layers to extract relevant features and reduce the dimensionality(along with conv layers).
#### 4) Dropout is used to ensure that the model does not overfit.
#### 5) Dense layers occur towards the end to generate the output.

In [2]:
# The CNN implemented 
def My_CNN(input_shape):

  # The sequential model, a linear class of layers, is used because the model is a sequence of layers
  model = Sequential()
  
  # Padding each item of the dataset with 2 columns and 2 rows of zeros for an image to be a (32,32,1) image
  model.add(ZeroPadding2D(padding=(2,2)))
  
  
  # Adding the first convolutional layer with 32 filters of size (3,3)
  # and activation=relu, padding=same
  model.add(Conv2D(32, kernel_size=(3,3), padding="same",  activation="relu", input_shape = input_shape))
  
  # BatchNormalization on the channel axis
  model.add(BatchNormalization(axis=-1))
  
  
  # Adding the 2nd convolutional layer with 32 filters of size (3,3)
  # and activation=relu, padding=same
  model.add(Conv2D(32, kernel_size=(3,3), activation="relu", padding="same"))
  
  # BatchNormalization on the channel axis
  model.add(BatchNormalization(axis=-1))
  
  
  #Adding the first max pooling layer with pool_size (2,2)
  # and stride_size=1
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  # Dropout to address overfitting
  model.add(Dropout(0.25))
            
    
  # Adding the 3rd convolutional layer with 64 filters of size (3,3)
  # and activation=relu, padding=same          
  model.add(Conv2D(64, kernel_size=(3,3), padding="same", activation="relu"))
  
  # BatchNormalization on the channel axis
  model.add(BatchNormalization(axis=-1))
  
  
  # Adding the 4th convolutional layer with 64 filters of size (3,3)
  # and activation=relu, padding=same
  model.add(Conv2D(64, kernel_size=(3,3), padding="same", activation="relu"))
  
  # BatchNormalization on the channel axis
  model.add(BatchNormalization(axis=-1))
  
  #Adding the 2nd max pooling layer with pool_size (2,2)
  model.add(MaxPooling2D(pool_size=(2,2)))
  
  # Dropout to address overfitting
  model.add(Dropout(0.25))
            
            
  # Unrolling into a vector to feed into an FC layer          
  model.add(Flatten())
  
  # The first layer with 512 neurons and actiation=relu
  model.add(Dense(512, activation="relu"))     
  
  
  model.add(BatchNormalization())
  model.add(Dropout(0.5))
  
  # Last softmax layer for class prediction
  model.add(Dense(4, activation="softmax"))
  
  return model
      

In [3]:
# Reading the file using pickle module
with open(("train_image.pkl"), 'rb') as train:
  df = pickle.load(train)

# Storing the pickled value as a numpy array
inputs = np.array(df)

# Reshaping the input images into a shape accepted by the LeNet5
inputs = np.array([i.reshape((28, 28, 1)) for i in inputs])

# Normalizing the input array
inputs = np.multiply(inputs, 1/255.0)

# Read the train_label file with pickle
with open(os.path.abspath('train_label.pkl'), 'rb') as train_label:
    outputs = pickle.load(train_label)
    
# One-hot encoding of the outputs obtained
outputs = pd.get_dummies(pd.Series(outputs)).values
print(outputs)

[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 ...
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]


In [4]:
# Check whether the shape we will be feeding is right
print("input_shape", inputs.shape)

input_shape (8000, 28, 28, 1)


In [5]:
# Call to the function with params as accepted by the input_shape parameter of a layer
model = My_CNN((inputs.shape,))

In [6]:
# Configure the model with an optimization process, an objective and metric(s)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

In [7]:
# Train the model and validate it over some epochs 
max_acc = 0

# Storing the model trained each time for plotting at a later stage
store_array = []
num_epochs = 20


# Train the model and store the best model in an h5 file
for i in range(num_epochs):
  
  history = model.fit(x=inputs, y=outputs, epochs=1, validation_split=0.1)
  store_array.append([history.history['acc'], history.history['val_acc'], history.history['loss'], history.history['val_loss']])
  
  if history.history['val_acc'][0] > max_acc:
    # Storing the best model 
    model.save('bestmodel.h5')
    # Choosing only that model whose validation accuracy is the best and using it for testing
    max_acc = history.history['val_acc'][0]
    


Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==============================] - 20s 3ms/step - loss: 0.6956 - acc: 0.7757 - val_loss: 2.6374 - val_acc: 0.2188TA: 
Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==============================] - 17s 2ms/step - loss: 0.4610 - acc: 0.8347 - val_loss: 2.6246 - val_acc: 0.3262
Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==============================] - 18s 2ms/step - loss: 0.3802 - acc: 0.8646 - val_loss: 0.8918 - val_acc: 0.6700
Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==============================] - 18s 3ms/step - loss: 0.3426 - acc: 0.8762 - val_loss: 1.2183 - val_acc: 0.5637 0s - loss: 0.3417 - 
Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==============================] - 18s 3ms/step - loss: 0.3094 - acc: 0.8857 - val_loss: 0.8686 - val_acc: 0.6987
Train on 7200 samples, validate on 800 samples
Epoch 1/1
7200/7200 [==

In [8]:
print("The maximum validation accuracy that could be obtained by a model:", max_acc, "\n\n\n")

# Displaying the parameters involved in each layer in "My_CNN" model
model.summary()

The maximum validation accuracy that could be obtained by a model: 0.8675 



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
________________________________________________________________

In [9]:
# Display created model
#SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [10]:
# Save the model into a json file
json_model = model.to_json()
with open('deb_lenet5.json', 'w') as model_json:
    model_json.write(json_model)

In [11]:
# Read the test file
with open(("test_image.pkl"), 'rb') as test:
  test_df = pickle.load(test)
  
# Store the pickled value into a numpy array
test_inputs = np.array(test_df)

# Reshaping for feeding into the network
test_inputs = np.array([i.reshape((28, 28, 1)) for i in test_inputs])

# Normalizing the input array
test_inputs = np.multiply(inputs, 1/255.0)

# Predicting the probability of an output for a given test example
pred_proba = model.predict(test_inputs)

#print(pred_proba)

# Choosing the label value which bears the highest probablility
pred_labels = np.argmax(pred_proba, axis=1)

In [12]:
# Checking the values of the first 10 results
print("Predicted labels of the first 10 test samples:", pred_labels[:10])

Predicted labels of the first 10 test samples: [3 3 3 3 3 3 3 3 3 3]


In [13]:
# A dictionary to map output labels obtained(one-hot encoded) to the right label
mapper = {
    0: 0,
    1: 2,
    2: 3,
    3: 6
}

pred_labels = np.array(list(map(lambda x: mapper[x], pred_labels)))
print(np.unique(pred_labels))

[6]


In [16]:
# Read the file for writing
submission_df = pd.DataFrame(columns=['Test_image_index', 'predicted_class'])


# Making a pandas series of the predicted labels with indexes
pred_labels_series = pd.Series(pred_labels, dtype='int32')
submission_df['predicted_class'] = pred_labels_series
submission_df['Test_image_index'] = pd.Series(list(range(len(pred_labels_series))))

# Identifying the 'Test_image_index' column as the index column
submission_df = submission_df.set_index(['Test_image_index'])

# displaying the first 20 predictions
print(submission_df.head(10))

# Saving to file "sample_submission.csv"
submission_df.to_csv(os.path.abspath('DebapriyaTula.csv'))

                  predicted_class
Test_image_index                 
0                               6
1                               6
2                               6
3                               6
4                               6
5                               6
6                               6
7                               6
8                               6
9                               6


In [15]:
# For accuracy evaluation on train and test examples

import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(store_array[:][0])
plt.plot(store_array[:][1])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(store_array[:][2])
plt.plot(store_array[:][3])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

<Figure size 640x480 with 1 Axes>

<Figure size 640x480 with 1 Axes>